<a href="https://colab.research.google.com/github/cbalkig/Anomaly_Detection_in_Videos/blob/master/train_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import listdir
from os.path import join, isdir
import numpy as np
from PIL import Image
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, TimeDistributed, Conv2D, Flatten, Dense, Dropout, Activation, MaxPooling2D, Activation
from keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Conv3D, BatchNormalization
from tensorflow.keras.optimizers import Adagrad
import matplotlib.pyplot as plt
import time
from numpy import save, load, asarray
from tensorflow.keras.optimizers import Adam
from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint, EarlyStopping
import uuid

In [ ]:
working_directory = '/content/drive/MyDrive/AnomalyDetectionInVideos'

In [ ]:
class Config:
    DATASET_PATH = os.path.join(working_directory, "files")
    EDIT_DATASET_PATH = os.path.join(working_directory, "edited_files")
    BATCH_SIZE = 128
    EPOCHS = 200
    MODEL_PATH = os.path.join(working_directory, "model.hdf5")
    MODEL_WEIGHTS_PATH = os.path.join(working_directory, "weights_MLP.h5")
    IMAGE_SIZE = 256

In [ ]:
def save_image(image_class, img_file_name, img):
    img_folder = join(Config.EDIT_DATASET_PATH, image_class)
    if not os.path.exists(img_folder):
        os.makedirs(img_folder)
    img_file = join(img_folder, img_file_name)
    img.save(img_file)

In [ ]:
def edit_images():
    for f in sorted(listdir(Config.DATASET_PATH)):
        directory_path = join(Config.DATASET_PATH, f)
        if isdir(directory_path):
            print("Processing", directory_path)
            count = 0
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    try:
                        image_class = str(int(int(str(c)[:-4]) / 20))
                        img = Image.open(img_path)
                        new_file_name = str(uuid.uuid4()) + ".tif"
                        save_image(image_class, new_file_name, img)
                    except:
                        print("Problem on file", img_path)

In [ ]:
edit_images()

In [ ]:
print("Sample Count", sum(len(files) for _, _, files in os.walk(Config.EDIT_DATASET_PATH)))
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=30, 
        fill_mode='nearest',
        brightness_range=[0.4,1.5],
        width_shift_range=0.2, 
        height_shift_range=0.2)
train_generator = train_datagen.flow_from_directory(
        Config.EDIT_DATASET_PATH,
        target_size=(Config.IMAGE_SIZE, Config.IMAGE_SIZE),
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        color_mode = 'grayscale',
        seed=10,
        class_mode='categorical')
print("Sample Count after ImageDataGenerator", train_generator.samples)
num_classes = len(os.listdir(Config.EDIT_DATASET_PATH))
print("Number of classes", num_classes)

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(10):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n, :, :, 0])
        plt.axis('off')
image_batch, label_batch = next(train_generator)
show_batch(image_batch, label_batch)

In [ ]:
def plot_acc_loss(trained):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(trained.epoch, trained.history["loss"], label="Train loss")
    ax[1].set_title('acc')
    ax[1].plot(trained.epoch, trained.history["accuracy"], label="Train acc")
    ax[0].legend()
    ax[1].legend()

In [ ]:
def train_model(reload_model=True):
    if reload_model:
        return load_model(Config.MODEL_PATH)
    input_shape = (Config.IMAGE_SIZE, Config.IMAGE_SIZE, 1)

    model = Sequential()

    #model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=input_shape))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=input_shape))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=input_shape))
    #model.add(MaxPooling2D(pool_size=(2, 2)))

    #model.add(Flatten())
    #model.add(Dense(64, activation="relu"))
    #model.add(Dropout(0.5))
    #model.add(Dense(num_classes, activation="softmax"))

    model.add(Conv2D(200, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(5, 5)))
    model.add(Conv2D(200, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(5, 5)))

    model.add(Flatten())
    model.add(Dense(180, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))

    model.summary()

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, epochs=Config.EPOCHS, verbose = 1)
    plot_acc_loss(history)
    return model

In [ ]:
model = train_model(reload_model=False)
print("Model is ready.")
model.save_weights(Config.MODEL_WEIGHTS_PATH)
model.save(Config.MODEL_PATH)
print("Model saved : ", Config.MODEL_PATH)